In [1]:
from scipy.stats import linregress
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
#"key"=api_key
from config import api_key

In [ ]:
# url = "https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/"
# params = {"appid":'212910'}

# response = requests.get(url, params=params).json()
# print(json.dumps(response, indent=4, sort_keys=True))

In [ ]:
# appid_df = pd.DataFrame()
# appid_df['Player Count'] = ""


# #Google Search for top rated STEAM video games 
# ids = ['1174180', '715950', '499450', '590380', '593380', '231200', '1085660', '653530' ]
# url = "https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/"
# index = 0
# for appid in action_df.l:
#     params = {"appid":appid}
    
#     #rl = "https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/"
#     response = requests.get(url, params=params).json()
#     try:
#         player_count = response['response']['player_count']
    
#     except Exception as e:
        
#         print("did not find count") 
        
#     appid_df.loc[index,'Player Count'] = player_count
    
#     index = index + 1
# # response = requests.get(url, params=params).json()
# #print(json.dumps(player_count, indent=4, sort_keys=True))
# appid_df

In [ ]:
#Returns games with this particular tag
# url = 'https://steamspy.com/api.php?request=tag&tag=Early+Access'
# response = requests.get(url)
# response.json()

In [2]:
#Returns games in the selected genre. Requires *genre* parameter. 
new_url = 'https://steamspy.com/api.php?request=genre&genre=action' 
response = requests.get(new_url).json()
response
action_df = pd.DataFrame(response).T
len(action_df)

24777

In [3]:
#Steps to create a clean data frame with all of the data from the action genre
#action_df = action_df.reset_index()
#del action_df['index']
#del action_df['level_0']
action_df.head()
#action_df.to_csv('Action_Genre_Data.csv', index=False)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
570,570,Dota 2,Valve,Valve,,1527142,315549,0,"200,000,000 .. 500,000,000",38515,1530,838,721,0,0,0,898826
730,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,,5930650,785631,0,"50,000,000 .. 100,000,000",29948,783,6352,259,0,0,0,869295
578080,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",,1183129,907693,0,"50,000,000 .. 100,000,000",23073,710,7307,201,0,0,0,387793
1063730,1063730,New World,Amazon Games,Amazon Games,,161726,74387,0,"50,000,000 .. 100,000,000",8135,1184,3489,466,3999,3999,0,50197
440,440,Team Fortress 2,Valve,Valve,,844927,57340,0,"50,000,000 .. 100,000,000",9650,1335,366,183,0,0,0,97407


In [26]:
#Create a player count Table using the AppID
app_count = [] 
appid_list = []
#Used to test the for loop21166:24777
# test_df = action_df.head()
#created to iterate through different portions of the action_df before connection to site errors out
set_df = action_df.iloc[21166:24777,0:2]

# search for player count by appid  
#url = "https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/"

for index, row in set_df.iterrows():
    appid = row['appid'] 
    name = row['name']
   
    print(f'checking index {index} AppId {appid} , Name {name}')
    
    params = {'appid': appid}
    url = "https://api.steampowered.com/ISteamUserStats/GetNumberOfCurrentPlayers/v1/"
    
    response = requests.get(url, params=params).json()
    try:
        player_count = response['response']['player_count']
    
    except Exception as e:
        
        print("did not find count") 
        
    app_count.append(player_count)
    appid_list.append(appid)

In [23]:
#appid_df = pd.DataFrame()
#print(f'{appid_list} , {app_count}') 
#len(appid_list)
#len(app_count)

972

In [6]:
#Create DF with appid and player count 
combineddata = {'appid':appid_list, 'player count':app_count}
combined_list_df = pd.DataFrame(combineddata)

combined_list_df

new_combined_df = pd.merge(combined_list_df,action_df, how="left", on=['appid','appid'])
new_combined_df.head()

,appid,player count,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,529100,0,Omni Link,Roencia,Roencia Game Creators,,36,4,0,"0 .. 20,000",0,0,0,0,399,399,0,0
1,513920,0,Desperate Times,Yokcos,Yokcos,,8,2,0,"0 .. 20,000",0,0,0,0,499,499,0,0
2,530740,0,Spareware,Rusto,Rusto,,20,7,0,"0 .. 20,000",0,0,0,0,0,0,0,0
3,527330,0,Grave: VR Prologue,"Broken Window Studios, LLC","Broken Window Studios, LLC",,5,12,0,"0 .. 20,000",0,0,0,0,799,799,0,0
4,529280,0,TEOT - The End OF Tomorrow,"Edsenses Creative Co., Ltd","Edsenses Creative Co., Ltd",,9,9,0,"0 .. 20,000",0,0,0,0,399,399,0,0


In [7]:
updated_data_types = {'average_forever': float, 'average_2weeks': float, 'median_forever': float, 'negative':float, 'positive': float,
                      'median_2weeks': float,'price': float, 'initialprice': float, 'discount': float, 'ccu': float}

new_combined_df = new_combined_df.astype(updated_data_types)
#new_combined_df.to_csv('AppID_PlayerCount_7.csv', index=False) 
new_combined_df

,appid,player count,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
0,529100,0,Omni Link,Roencia,Roencia Game Creators,,36.0,4.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,399.0,399.0,0.0,0.0
1,513920,0,Desperate Times,Yokcos,Yokcos,,8.0,2.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,499.0,499.0,0.0,0.0
2,530740,0,Spareware,Rusto,Rusto,,20.0,7.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,527330,0,Grave: VR Prologue,"Broken Window Studios, LLC","Broken Window Studios, LLC",,5.0,12.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,799.0,799.0,0.0,0.0
4,529280,0,TEOT - The End OF Tomorrow,"Edsenses Creative Co., Ltd","Edsenses Creative Co., Ltd",,9.0,9.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,399.0,399.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3738,544380,0,Tesla: The Weather Man,"Thoughtquake Studios, LLC","Thoughtquake Studios, LLC",,6.0,1.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,299.0,299.0,0.0,0.0
3739,831520,0,Jurassic City Walk,Real Fighting,Real Fighting,,4.0,21.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3740,834340,0,The Hardest Dungeon,Pippala Games,Pippala Games,,16.0,4.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3741,818670,0,Save Snegurochka!,TIPA Games Berlin,TIPA Games Berlin,,5.0,0.0,0,"0 .. 20,000",0.0,0.0,0.0,0.0,899.0,899.0,0.0,0.0


In [27]:
#read in each of the csv files created
appid_playercount = pd.read_csv('./AppID_PlayerCount.csv')
appid_playercount2 = pd.read_csv('./AppID_PlayerCount_2.csv')
appid_playercount3 = pd.read_csv('./AppID_PlayerCount_3.csv')
appid_playercount4 = pd.read_csv('./AppID_PlayerCount_4.csv')
appid_playercount5 = pd.read_csv('./AppID_PlayerCount_5.csv')
appid_playercount6 = pd.read_csv('./AppID_PlayerCount_6.csv')
appid_playercount7 = pd.read_csv('./AppID_PlayerCount_7.csv')

In [33]:
final_combined_df = pd.concat([appid_playercount,appid_playercount2, appid_playercount3, appid_playercount4, 
            appid_playercount5, appid_playercount6, appid_playercount7])

#final_combined_df.to_csv('Final_Combined_Data.csv', index=False)
final_combined_df.dtypes

appid                int64
player count         int64
name                object
developer           object
publisher           object
score_rank         float64
positive           float64
negative           float64
userscore            int64
owners              object
average_forever    float64
average_2weeks     float64
median_forever     float64
median_2weeks      float64
price              float64
initialprice       float64
discount           float64
ccu                float64
dtype: object

In [40]:
final_combined_df = final_combined_df.sort_values('player count', ascending=False)
final_combined_df.head()

,appid,player count,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu
1,730,378812,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,NaN,5926012.0,785171.0,0,"50,000,000 .. 100,000,000",30075.0,775.0,6864.0,264.0,0.0,0.0,0.0,747905.0
0,570,368142,Dota 2,Valve,Valve,NaN,1522243.0,314225.0,0,"200,000,000 .. 500,000,000",38004.0,1697.0,752.0,953.0,0.0,0.0,0.0,624634.0
12,1599340,162724,Lost Ark,Smilegate RPG,Amazon Games,NaN,128376.0,45481.0,0,"20,000,000 .. 50,000,000",2927.0,1598.0,673.0,499.0,0.0,0.0,0.0,210671.0
4,440,97140,Team Fortress 2,Valve,Valve,NaN,844446.0,57326.0,0,"50,000,000 .. 100,000,000",8822.0,694.0,338.0,104.0,0.0,0.0,0.0,97510.0
10,252490,71468,Rust,Facepunch Studios,Facepunch Studios,NaN,728823.0,111648.0,0,"20,000,000 .. 50,000,000",20162.0,1394.0,3417.0,720.0,2679.0,3999.0,33.0,80990.0


In [ ]:
# appid_info = []
# app_name = []
# avg_play_total_time = []
# avg_median_total = []
# avg_play_time_2weeks = []
# median_2Weeks = []
# price = []

In [ ]:
# #by App ID 
# list_ids = ['570','730', '578080','590380', '593380', '231200', '1085660', '653530', '1174180', '715950', '499450',]
# #new_url = 'https://steamspy.com/api.php?request=appdetails&appid='
          
# #params = {"appid":ids}
# for value in list_ids:
#     url= f'https://steamspy.com/api.php?request=appdetails&appid={value}'
   
#     response = requests.get(url).json()   
   
#     appid_info.append(response['appid'])
#     app_name.append(response['name'])
#     avg_play_total_time.append(response['average_forever'])
#     avg_median_total.append(response['median_forever'])
#     avg_play_time_2weeks.append(response['average_2weeks'])
#     median_2Weeks.append(response['median_2weeks'])
#     price.append(response['price'])

# print(app_name)   

#print(price)

In [ ]:

# steamdata = {'APP ID':appid_info,'App Name': app_name, 'Avg Total Play Time': avg_play_total_time,'Median Total Play Time': avg_median_total, 
#              'Avg Play Time 2 Weeks': avg_play_time_2weeks, 'Median 2 Weeks':median_2Weeks, 'Price':price
#             }
# # steam_df['APP ID'] = appid_info
# # steam_df['App Name'] = app_name
# # steam_df['Avg Play Time'] = avg_play_total_time
# # steam_df['Median Play Time'] =  avg_median_total
# # steam_df['Avg Play Time 2 Weeks'] = avg_play_time_2weeks
# # steam_df['Median 2 Weeks'] = median_2Weeks
# steam_df = pd.DataFrame(steamdata)
    
# steam_df

In [44]:
#Returns the Top 100 apps by players in the last two weeks
new_url = 'https://steamspy.com/api.php?request=top100in2weeks'
steam_df = pd.DataFrame()
steam_df['APP ID'] = ""
steam_df['App Name'] = ""
steam_df['Avg Play Time'] = ""
steam_df['Avg Play Time 2 Weeks'] = ""

# new_url = 'https://steamspy.com/api.php?request=top100in2weeks'
# params = { '?request=': 'top50', "length":"in2weeks"}
response = requests.get(new_url).json()
# response. weather_df.to_csv('WeatherPy.csv', index=False) 
#top100_df = pd.DataFrame(response).T
#top100_df
#top100_df.to_csv('Top100.csv', index=False) 

In [43]:
#top100_df.dtypes

In [ ]:
#return apps with 1,000-1,999 owners.
new_url = 'https://steamspy.com/api.php?request=all&page=1'
steam_df = pd.DataFrame()
steam_df['APP ID'] = ""
steam_df['App Name'] = ""
steam_df['Avg Play Time'] = ""
steam_df['Avg Play Time 2 Weeks'] = ""

# new_url = 'https://steamspy.com/api.php?request=top100in2weeks'
# params = { '?request=': 'top50', "length":"in2weeks"}
response = requests.get(new_url).json()
# response. weather_df.to_csv('WeatherPy.csv', index=False) 
owners_df = pd.DataFrame(response).T
owners_df = owners_df.reset_index()
del owners_df['index']
owners_df
#response
owners_df.to_csv('SEAMS_Data.csv', index=False)
#weather_df.to_csv('WeatherPy.csv', index=False) 

In [ ]:
# new_url = 'https://steamspy.com/api.php?request=top100in2weeks'
# new_url2 = 'https://steamspy.com/api.php?request=appdetails&appid=231200'
# steam_df = pd.DataFrame()
# steam_df['APP ID'] = ""
# steam_df['App Name'] = ""
# steam_df['Avg Play Time'] = ""
# steam_df['Median Play Time'] = ""
# steam_df['Avg Play Time 2 Weeks'] = ""
# steam_df['Median Play Time 2 Weeks'] = ""
# # new_url = 'https://steamspy.com/api.php?request=top100in2weeks'
# # params = { '?request=': 'top50', "length":"in2weeks"}
# response = requests.get(new_url2).json()
# response 

Final DataFrame and Group Code Starts Here

In [41]:
final_combined_data_df = pd.read_csv('./Final_Combined_DataF.csv')

In [42]:
final_combined_data_df

,appid,player count,name,developer,publisher,score_rank,positive,negative,userscore,owners,...,price,initialprice,discount,ccu,Final Price_$,Initial Price_$,Discount_$,Total Score,Mark,Tier
0,570,368142,Dota 2,Valve,Valve,NaN,1522243,314225,0,"200,000,000 .. 500,000,000",...,0,0,0,624634,9.53,0.00,0.00,1836468,83%,A
1,730,378812,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,NaN,5926012,785171,0,"50,000,000 .. 100,000,000",...,0,0,0,747905,2.64,0.00,0.00,6711183,88%,B
2,578080,67089,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",NaN,1182394,907360,0,"50,000,000 .. 100,000,000",...,0,0,0,256666,2.22,0.00,0.00,2089754,57%,B
3,1063730,40836,New World,Amazon Games,Amazon Games,NaN,161467,74422,0,"50,000,000 .. 100,000,000",...,3999,3999,0,52597,0.94,39.99,39.99,235889,68%,B
4,440,97140,Team Fortress 2,Valve,Valve,NaN,844446,57326,0,"50,000,000 .. 100,000,000",...,0,0,0,97510,1.04,0.00,0.00,901772,94%,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21114,544380,0,Tesla: The Weather Man,"Thoughtquake Studios, LLC","Thoughtquake Studios, LLC",NaN,6,1,0,"0 .. 20,000",...,299,299,0,0,0.00,2.99,2.99,7,86%,M
21115,831520,0,Jurassic City Walk,Real Fighting,Real Fighting,NaN,4,21,0,"0 .. 20,000",...,0,0,0,0,0.00,0.00,0.00,25,-84%,M
21116,834340,0,The Hardest Dungeon,Pippala Games,Pippala Games,NaN,16,4,0,"0 .. 20,000",...,0,0,0,0,0.00,0.00,0.00,20,80%,M
21117,818670,0,Save Snegurochka!,TIPA Games Berlin,TIPA Games Berlin,NaN,5,0,0,"0 .. 20,000",...,899,899,0,0,0.00,8.99,8.99,5,100%,M
